# Layer by Layer approach for bulding a MLP (Multi-Layer Perseptron) Model

In [27]:
import numpy as np
from tqdm import tqdm  # Import tqdm for progress bars

# Base layer class: A generic layer in the neural network
class Layer:
    def __init__(self, input, output):
        """
        Initialize the Layer class.

        Parameters:
        input (np.ndarray): Input data to this layer.
        output (np.ndarray): Output data from this layer.
        """
        self.input = input   # Stores the input to this layer (type: np.ndarray)
        self.output = output # Stores the output from this layer (type: np.ndarray)

    def forward(self, output):
        """
        Defines the forward pass through the layer.

        This method should be implemented by subclasses to specify how the input data
        is transformed into output data using the layer's parameters.

        Parameters:
        output (np.ndarray): Input data to be processed by the layer.

        Returns:
        np.ndarray: Output data after applying the layer's transformation.
        """
        pass  # To be implemented by subclasses, defines how the input is transformed to output

    def backward(self, output_gradient, learning_rate):
        """
        Defines the backward pass through the layer.

        This method should be implemented by subclasses to specify how the gradients
        of the loss function with respect to the output are used to update the layer's
        parameters and compute the gradient with respect to the input data.

        Parameters:
        output_gradient (np.ndarray): Gradient of the loss function with respect to the layer's output.
        learning_rate (float): Learning rate for updating the layer's parameters.

        Returns:
        np.ndarray: Gradient of the loss function with respect to the layer's input data.
        """
        pass  # To be implemented by subclasses, defines how gradients are used to update parameters

# Dense layer class: A fully connected layer
class Dense(Layer):
    def __init__(self, no_of_perseptrons):
        """
        Initialize the Dense layer with a given number of neurons.

        Parameters:
        output_size (int): Number of neurons (perceptrons) in this layer.
        """
        self.no_of_perseptrons = no_of_perseptrons  # Number of neurons in the layer (type: int)
        self.input_size = None  # Input size will be determined during the forward pass (type: int or None)
        self.weights = None  # Weights for the layer's connections (type: np.ndarray or None)
        self.bias = None  # Biases for the layer (type: np.ndarray or None)
        self.output_size = no_of_perseptrons

        #todo: add logic to change  none types to actual values

    def forward(self, input):
        """
        Perform the forward pass through the dense layer.

        Computes the output of the dense layer by applying a linear transformation to the
        input data using the layer's weights and biases.

        Parameters:
        input (np.ndarray): Input data to the layer (shape: [input_size, 1]).

        Returns:
        np.ndarray: Output data from the layer after applying the linear transformation (shape: [output_size, 1]).
        """
        if input.ndim == 1:
            input = input.reshape(-1, 1)
        elif input.ndim > 2:
            raise ValueError("Input must be 1D or 2D array")

        if self.input_size is None:
            self.input_size = input.shape[0]
            self.weights = np.random.randn(self.no_of_perseptrons, self.input_size) * np.sqrt(2.0 / (self.input_size + self.no_of_perseptrons))
            self.bias = np.zeros((self.no_of_perseptrons, 1))

        self.input = input
        self.output = np.dot(self.weights, self.input) + self.bias
        return self.output

    def updater(self, input_size):
        """
        Initialize the weights and biases for the dense layer.

        Parameters:
        input_size (int): Number of input features.
        """
        self.input_size = input_size
        self.output_size = self.no_of_perseptrons

    def backward(self, output_gradient, learning_rate):
        """
        Perform the backward pass through the dense layer and update its parameters.

        Computes the gradients of the loss function with respect to the layer's weights
        and biases, updates these parameters using the provided learning rate, and calculates
        the gradient with respect to the input data.

        Parameters:
        output_gradient (np.ndarray): Gradient of the loss function with respect to the layer's output (shape: [output_size, 1]).
        learning_rate (float): Learning rate for updating the weights and biases.

        Returns:
        np.ndarray: Gradient of the loss function with respect to the input data (shape: [input_size, 1]).
        """
        if output_gradient.ndim == 1:
            output_gradient = output_gradient.reshape(-1, 1)

        self.old_weights = self.weights
        weights_gradient = np.dot(output_gradient, self.input.T)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        input_gradient = np.dot(self.old_weights.T, output_gradient)
        return input_gradient.reshape(self.input.shape)

# Activation class: Applies an activation function
class Activation:
    def __init__(self, activation, activation_prime):
        """
        Initialize the Activation class with a specific activation function and its derivative.

        Parameters:
        activation (callable): Activation function to be applied (e.g., sigmoid, ReLU).
        activation_prime (callable): Derivative of the activation function.
        """
        self.activation = activation  # Activation function to be used (type: callable)
        self.activation_prime = activation_prime  # Derivative of the activation function (type: callable)

    def forward(self, input):
        """
        Perform the forward pass through the activation function.

        Applies the activation function to the input data and stores the result.

        Parameters:
        input (np.ndarray): Input data to the activation function (type: np.ndarray).

        Returns:
        np.ndarray: Output data after applying the activation function (type: np.ndarray).
        """
        self.input = input  # Save input for use in the backward pass (type: np.ndarray)
        return self.activation(self.input)  # Apply activation function (type: np.ndarray)

    def backward(self, output_gradient, learning_rate):
        """
        Perform the backward pass through the activation function.

        Computes the gradient of the loss function with respect to the input data
        by applying the derivative of the activation function.

        Parameters:
        output_gradient (np.ndarray): Gradient of the loss function with respect to the activation function's output (type: np.ndarray).
        learning_rate (float): Learning rate (not used in this method, but included for consistency).

        Returns:
        np.ndarray: Gradient of the loss function with respect to the input data (type: np.ndarray).
        """
        return np.multiply(output_gradient, self.activation_prime(self.input))  # Compute gradient with respect to the input data (type: np.ndarray)

# Sigmoid activation class
class Sigmoid(Activation):
    def __init__(self):
        """
        Initialize the Sigmoid activation function and its derivative.
        """
        # Define sigmoid function and its derivative
        sigmoid = staticmethod(lambda x: 1 / (1 + np.exp(-x)))  # Sigmoid function (type: callable)
        sigmoid_prime = staticmethod(lambda x: (1 / (1 + np.exp(-x))) * (1 - (1 / (1 + np.exp(-x)))))  # Derivative of sigmoid function (type: callable)
        super().__init__(sigmoid, sigmoid_prime)  # Initialize parent class with sigmoid functions

# Loss class: Provides loss function and its derivative
class Loss(Layer):
    @staticmethod
    def mse(y_true, y_pred):
        """
        Calculate the Mean Squared Error (MSE) loss between the true labels and predicted labels.

        Parameters:
        y_true (np.ndarray): True labels (shape: [n_samples, 1]).
        y_pred (np.ndarray): Predicted labels (shape: [n_samples, 1]).

        Returns:
        float: MSE loss value indicating the average squared difference between true and predicted labels.
        """
        return np.mean(np.power(y_true - y_pred, 2))  # Mean Squared Error loss (type: float)

    @staticmethod
    def mse_derivative(y_true, y_pred):
        """
        Calculate the derivative of the Mean Squared Error (MSE) loss function with respect to the predictions.

        Parameters:
        y_true (np.ndarray): True labels (shape: [n_samples, 1]).
        y_pred (np.ndarray): Predicted labels (shape: [n_samples, 1]).

        Returns:
        np.ndarray: Gradient of MSE loss with respect to predictions (shape: [n_samples, 1]).
        """
        return 2 * (y_pred - y_true) / np.size(y_true)  # Derivative of MSE loss (type: np.ndarray)

# CometNet class: Manages the neural network
class CometNet:
    def __init__(self, network, X, y, epochs=10, learning_rate=0.1, input_size=None):
        """
        Initialize the CometNet class for training and predicting with a neural network.

        Parameters:
        network (list): List of layers in the network (each layer is an instance of a Layer subclass).
        x_train (list): Training input data (list of np.ndarray).
        y_train (list): Training target data (list of np.ndarray).
        epochs (int): Number of training epochs (default: 1000).
        learning_rate (float): Learning rate for updating the parameters (default: 0.01).
        """
        self.network = network
        self.X = X
        self.y = y
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.input_size = input_size # Store the input size

        # Initialize weights and biases for each layer if input_size is provided
        if self.input_size:
            for layer in self.network:
                if isinstance(layer, Dense):
                    layer.updater(self.input_size)
                    self.input_size = layer.output_size # Update input size for the next layer

    def predict(self, input):
        """
        Perform a forward pass through the network to make predictions.

        Applies the forward pass through each layer in the network to generate predictions
        from the input data.

        Parameters:
        input (np.ndarray): Input data to the network (shape: [input_size, 1]).

        Returns:
        np.ndarray: Output predictions from the network after passing through all layers (shape: [output_size, 1]).
        """
        output = input  # Start with the input data (type: np.ndarray)
        for layer in self.network:  # Pass input through each layer in the network
            output = layer.forward(output)  # Apply each layer's forward pass (type: np.ndarray)
        return output  # Return final output (type: np.ndarray)

    def train(self):
        """
        Train the neural network using the provided training data.

        Iterates over the specified number of epochs, performing forward and backward passes
        through the network for each training example. Updates the parameters of the network
        based on the computed gradients.

        This method prints the progress of training and updates the network parameters after
        each epoch.
        """
        for epoch in range(self.epochs):  # Iterate through each epoch (type: int)
            error = 0  # Initialize error for this epoch (type: float)
            print(f"Epoch {epoch + 1}/{self.epochs} started.")  # Debugging print statement
            for x, y in tqdm(zip(self.X, self.y), total=len(self.X), desc=f"Epoch {epoch + 1}"):  # Iterate through training data with progress bar
                # Ensure x is a 2D array
                x = x.reshape(-1, 1) if x.ndim == 1 else x

                # Ensure y is a 2D array with shape (1, 1)
                y = np.array([[y]], dtype=float) if np.isscalar(y) else y.reshape(1, -1)

                output = self.predict(x)  # Compute network output for the current input (type: np.ndarray)
                error += Loss.mse(y, output)  # Calculate error using the Mean Squared Error loss function (type: float)
                output_gradient = Loss.mse_derivative(y, output)  # Compute gradient of the loss with respect to the output (type: np.ndarray)
                for layer in reversed(self.network):  # Update parameters starting from the last layer
                    output_gradient = layer.backward(output_gradient, self.learning_rate)  # Perform backward pass and update parameters (type: np.ndarray)
            error /= len(self.X)  # Compute average error over all training examples (type: float)
            print(f"Epoch {epoch + 1} completed with average error: {error:.6f}")  # Print average error for the epoch


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.utils import resample

class DataPreprocessor:
    """
    A class for preprocessing network traffic data.
    """

    def __init__(self, datetime_col='Timestamp', label_col='Label', random_state=42):
        """
        Initialize the DataPreprocessor.

        Args:
            datetime_col (str): Name of the datetime column.
            label_col (str): Name of the label column.
            random_state (int): Random seed for reproducibility.
        """
        self.datetime_col = datetime_col
        self.label_col = label_col
        self.random_state = random_state
        self.label_encoder = LabelEncoder()
        self.scaler = MinMaxScaler()
        self.imputer = SimpleImputer(strategy='mean')

    def read_and_combine_data(self, file_paths):
        """
        Read CSV files and combine them into a single DataFrame.

        Args:
            file_paths (list): List of paths to the CSV files.

        Returns:
            pd.DataFrame: Combined DataFrame.
        """
        df_list = [pd.read_csv(file_path, encoding='latin1') for file_path in file_paths]
        df = pd.concat(df_list).reset_index(drop=True)
        df.columns = df.columns.str.strip().str.replace(' ', '_')
        return df

    def preprocess_data(self, df):
        """
        Preprocess the data by encoding labels, handling timestamps, and dropping unnecessary columns.

        Args:
            df (pd.DataFrame): Input DataFrame.

        Returns:
            pd.DataFrame: Preprocessed DataFrame.
        """
        # Encode labels
        df[self.label_col] = self.label_encoder.fit_transform(df[self.label_col])
        df[self.label_col] = df[self.label_col].apply(lambda x: 0 if x == 0 else 1)

        # Handle timestamp
        if self.datetime_col in df.columns:
            df[self.datetime_col] = pd.to_datetime(df[self.datetime_col], errors='coerce')
            df.dropna(subset=[self.datetime_col], inplace=True)
            df['minutes_from_midnight'] = (df[self.datetime_col].dt.hour * 60 +
                                           df[self.datetime_col].dt.minute +
                                           df[self.datetime_col].dt.second / 60 +
                                           df[self.datetime_col].dt.microsecond / 60000000)
            df.drop(columns=[self.datetime_col], inplace=True)

        # Drop unnecessary columns
        columns_to_drop = ['Flow_ID', 'Source_IP', 'Destination_IP']
        df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

        return df

    def resample_data(self, df, proportions):
        """
        Resample the data to achieve desired class proportions.

        Args:
            df (pd.DataFrame): Input DataFrame.
            proportions (list): Desired proportions for each label.

        Returns:
            pd.DataFrame: Resampled DataFrame.
        """
        df_majority = df[df[self.label_col] == 0]
        df_minority = df[df[self.label_col] == 1]

        n_samples_majority = int(len(df) * proportions[0])
        n_samples_minority = int(len(df) * proportions[1])

        df_majority_resampled = resample(df_majority, replace=False, n_samples=n_samples_majority, random_state=self.random_state)
        df_minority_resampled = resample(df_minority, replace=True, n_samples=n_samples_minority, random_state=self.random_state)

        return pd.concat([df_majority_resampled, df_minority_resampled])

    def select_features(self, df):
        """
        Select features based on a predefined list.

        Args:
            df (pd.DataFrame): Input DataFrame.

        Returns:
            pd.DataFrame: DataFrame with selected features.
        """
        selected_features = [
            'Source_Port', 'Destination_Port', 'Protocol', 'Flow_Duration', 'Fwd_Packet_Length_Max',
            'Fwd_Packet_Length_Min', 'Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Std',
            'Bwd_Packet_Length_Max', 'Bwd_Packet_Length_Min', 'Bwd_Packet_Length_Mean',
            'Bwd_Packet_Length_Std', 'Flow_IAT_Mean', 'Flow_IAT_Std', 'Flow_IAT_Max',
            'Fwd_IAT_Total', 'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max', 'Bwd_IAT_Std',
            'Bwd_IAT_Max', 'Fwd_PSH_Flags', 'Bwd_Packets/s', 'Min_Packet_Length',
            'Max_Packet_Length', 'Packet_Length_Mean', 'Packet_Length_Std',
            'Packet_Length_Variance', 'FIN_Flag_Count', 'SYN_Flag_Count', 'ACK_Flag_Count',
            'URG_Flag_Count', 'Down/Up_Ratio', 'Average_Packet_Size', 'Avg_Fwd_Segment_Size',
            'Avg_Bwd_Segment_Size', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward',
            'Idle_Mean', 'Idle_Max', 'Idle_Min', 'minutes_from_midnight'
        ]
        return df[selected_features + [self.label_col]]

    def scale_and_impute(self, X_train, X_test):
        """
        Scale features and impute missing values.

        Args:
            X_train (np.array): Training feature set.
            X_test (np.array): Test feature set.

        Returns:
            tuple: Scaled and imputed training and test sets.
        """
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        X_train_imputed = self.imputer.fit_transform(X_train_scaled)
        X_test_imputed = self.imputer.transform(X_test_scaled)

        return X_train_imputed, X_test_imputed

    def process_data(self, file_paths, proportions, verbose=False):
        """
        Process the data through all preprocessing steps.

        Args:
            file_paths (list): List of paths to the CSV files.
            proportions (list): Desired proportions for each label.
            verbose (bool): Whether to print additional information.

        Returns:
            tuple: Processed training and test sets (X_train, X_test, y_train, y_test).
        """
        df = self.read_and_combine_data(file_paths)
        df = self.preprocess_data(df)
        df_resampled = self.resample_data(df, proportions)
        df_filtered = self.select_features(df_resampled)

        y = df_filtered[self.label_col]
        X = df_filtered.drop(columns=[self.label_col])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=self.random_state)
        X_train_imputed, X_test_imputed = self.scale_and_impute(X_train, X_test)

        if verbose:
            print("Final shapes:")
            print("X_train shape:", X_train_imputed.shape)
            print("X_test shape:", X_test_imputed.shape)
            print("y_train shape:", y_train.shape)
            print("y_test shape:", y_test.shape)

        return X_train_imputed, X_test_imputed, y_train, y_test

In [3]:
# Define file paths
train_file_paths = [
    r'/content/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
    r'/content/Monday-WorkingHours.pcap_ISCX.csv',
    r'/content/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv'
]
test_file_paths = [
    r'/content/Wednesday-workingHours.pcap_ISCX.csv'
]

# Define desired proportions for each label
proportions = [0.4, 0.6]

# Initialize DataPreprocessor
preprocessor = DataPreprocessor(datetime_col='Timestamp', label_col='Label', random_state=42)

# Process training data
print("Processing training data:")
X_train, X_test, y_train, y_test = preprocessor.process_data(train_file_paths, proportions, verbose=False)

# Process test data
print("\nProcessing test data:")
X_train_test, X_test_test, y_train_test, y_test_test = preprocessor.process_data(test_file_paths, proportions, verbose=True)

Processing training data:


<ipython-input-2-6461e323bdee>:39: DtypeWarning: Columns (0,1,3,6,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(file_path, encoding='latin1') for file_path in file_paths]



Processing test data:
Final shapes:
X_train shape: (554161, 42)
X_test shape: (138541, 42)
y_train shape: (554161,)
y_test shape: (138541,)


In [4]:
X_train[1].shape

(42,)

In [28]:
X_train = np.array(X_train)  # Ensure X_train is a numpy array
y_train = np.array(y_train)  # Ensure y_train is a numpy array
# Define the network
network = [
    Dense(100),
    Sigmoid(),
    Dense(200),
    Sigmoid(),
    Dense(1),
    Sigmoid()
]

comet_net = CometNet(network, X_train, y_train, epochs=2, learning_rate=0.1, input_size=X_train.shape[1]) # Pass input size to the network
comet_net.train()



Epoch 1/2 started.


Epoch 1:   0%|          | 0/316888 [00:00<?, ?it/s]


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [10]:
X_test = np.array(X_test)
# test the model
y_pred = comet_net.predict(X_test)
print(y_pred)

# check confution matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

# classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


ValueError: shapes (100,42) and (79222,42) not aligned: 42 (dim 1) != 79222 (dim 0)